# Tahoe Regional Transportation Plan Forecasting
> Data Engineering Tasks
* Residential development forecasting for 2035 and 2050

## Setup

In [ ]:
# import packages
import pandas as pd
import pathlib
from pathlib import Path
import os
import arcpy
from utils import *
import numpy as np
import pickle
# external connection packages
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

# pandas options
pd.options.mode.copy_on_write = True
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
pd.options.display.max_rows    = 999

# my workspace 
workspace = r"C:\Users\mbindl\Desktop\Workspace.gdb"
# current working directory
local_path = pathlib.Path().absolute()
# set data path as a subfolder of the current working directory TravelDemandModel\2022\
data_dir = local_path.parents[0] / 'data'
# folder to save processed data
out_dir  = local_path.parents[0] / 'data/processed_data'
# workspace gdb for stuff that doesnt work in memory
# gdb = os.path.join(local_path,'Workspace.gdb')
gdb = workspace
# set environement workspace to in memory 
arcpy.env.workspace = 'memory'
# # clear memory workspace
# arcpy.management.Delete('memory')

# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)

# get parcels from the database
# network path to connection files
filePath = "F:/GIS/PARCELUPDATE/Workspace/"
# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")
sdeEdit    = os.path.join(filePath, "Edit.sde")

# Pickle variables
# part 1 - spatial joins and new categorical fields
parcel_pickle_part1    = data_dir / 'parcel_pickle1.pkl'
# part 2 - forecasting applied
parcel_pickle_part2    = data_dir / 'parcel_pickle2.pkl'

# columsn to list
initial_columns = [ 'APN',
                    'APO_ADDRESS',
                    'Residential_Units',
                    'TouristAccommodation_Units',
                    'CommercialFloorArea_SqFt',
                    'YEAR',
                    'JURISDICTION',
                    'COUNTY',
                    'OWNERSHIP_TYPE',
                    'COUNTY_LANDUSE_DESCRIPTION',
                    'EXISTING_LANDUSE',
                    'REGIONAL_LANDUSE',
                    'YEAR_BUILT',
                    'PLAN_ID',
                    'PLAN_NAME',
                    'ZONING_ID',
                    'ZONING_DESCRIPTION',
                    'TOWN_CENTER',
                    'LOCATION_TO_TOWNCENTER',
                    'TAZ',
                    'PARCEL_ACRES',
                    'PARCEL_SQFT',
                    'WITHIN_BONUSUNIT_BNDY',
                    'WITHIN_TRPA_BNDY',
                    'SHAPE']

### Functions

In [ ]:
def get_conn(db):
    # Get database user and password from environment variables on machine running script
    db_user             = os.environ.get('DB_USER')
    db_password         = os.environ.get('DB_PASSWORD')
    # driver is the ODBC driver for SQL Server
    driver              = 'ODBC Driver 17 for SQL Server'
    # server names are
    sql_12              = 'sql12'
    sql_14              = 'sql14'
    # make it case insensitive
    db = db.lower()
    # make sql database connection with pyodbc
    if db   == 'sde_tabular':
        connection_string = f"DRIVER={driver};SERVER={sql_12};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    elif db == 'tahoebmpsde':
        connection_string = f"DRIVER={driver};SERVER={sql_14};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    elif db == 'sde':
        connection_string = f"DRIVER={driver};SERVER={sql_12};DATABASE={db};UID={db_user};PWD={db_password}"
        connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})
        engine = create_engine(connection_url)
    # else return None
    else:
        engine = None
    # connection file to use in pd.read_sql
    return engine

# save to pickle
def to_pickle(data, filename):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f'{filename} pickled')

# save to pickle and feature class
def to_pickle_fc(data, filename):
    data.spatial.to_featureclass(filename)
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f'{filename} pickled and saved as feature class')

# get a pickled file as a dataframe
def from_pickle(filename):
    with open(filename, 'rb') as f:
        data = pickle.load(f)
    print(f'{filename} unpickled')
    return data
def get_commercial_zones(df):
    columns_to_keep = ['Zoning_ID', 'Category', 'Density']
    # filter Use_Type to Multiple Family Dwelling
    df = df.loc[df['Category'] == 'Commercial']
    return df[columns_to_keep]

def get_tourist_zones(df):
    columns_to_keep = ['Zoning_ID', 'Category', 'Density']
    # filter Use_Type to Multiple Family Dwelling
    df = df.loc[df['Category'] == 'Tourist Accommodation']
    return df[columns_to_keep]

# function to get where Zoningin_ID Use_Type = Multi-Family and Density
def get_mf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Multiple Family Dwelling
    df = df.loc[df['Use_Type'] == 'Multiple Family Dwelling']
    return df[columns_to_keep]

# function to get where Zoningin_ID Use_Type = Multi-Family and Density
def get_sf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Multiple Family Dwelling
    df = df.loc[df['Use_Type'] == 'Single Family Dwelling']
    return df[columns_to_keep]

def get_mf_only_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Single Family Dwelling and not Multiple Family Dwelling
    dfMF = get_mf_zones(df)
    dfSF = get_sf_zones(df)
    # get Zoning_ID that are in both dataframes
    df = dfMF.loc[~dfMF['Zoning_ID'].isin(dfSF['Zoning_ID'])]
    return df[columns_to_keep]

def get_sf_only_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # filter Use_Type to Single Family Dwelling and not Multiple Family Dwelling
    dfMF = get_mf_zones(df)
    dfSF = get_sf_zones(df)
    df = dfSF.loc[~dfSF['Zoning_ID'].isin(dfMF['Zoning_ID'])]
    return df[columns_to_keep]

def get_sf_mf_zones(df):
    columns_to_keep = ['Zoning_ID', 'Use_Type', 'Density']
    # get SF and MF zones
    dfSF = get_sf_zones(df)
    dfMF = get_mf_zones(df)
    # add the two dataframes together
    df = pd.concat([dfSF, dfMF])
    # only keep duplicate Zoning_ID
    df = df[df.duplicated(subset=['Zoning_ID'], keep=False)]
    return df[columns_to_keep]

def get_recieving_zones(df):
    columns_to_keep = ['Zoning_ID', 'SPECIAL_DESIGNATION']
    # filter transfer recieving
    df = df.loc[df['SPECIAL_DESIGNATION'] == 'Receive']
    return df[columns_to_keep]

def get_sending_zones(df):
    columns_to_keep = ['Zoning_ID', 'SPECIAL_DESIGNATION']
    df = df.loc[df['SPECIAL_DESIGNATION'] == 'Transfer']
    return df[columns_to_keep]

def forecast_residential_units(df, condition, target_sum, reason):
    # filter to parcels available for development
    sdfAvailable = df.loc[eval(condition)]
    running_sum = 0
    rows_to_fill = []
    # Loop through the rows and fill the 'new_column'
    for idx, row in sdfAvailable.iterrows():
        # Calculate the remaining amount that can be filled
        remaining_amount = target_sum - running_sum
        if row['MAX_UNITS'] <= remaining_amount:
            # If the current row's value fits, add it to the column
            df.loc[idx, 'FORECASTED_RESIDENTIAL_UNITS'] = row['MAX_UNITS']
            running_sum += row['MAX_UNITS']
            if row['MAX_UNITS'] > 0:
                rows_to_fill.append(idx)
        elif remaining_amount > 0:
            # If it exceeds the remaining amount, fill with the remaining value
            df.loc[idx, 'FORECASTED_RESIDENTIAL_UNITS'] = remaining_amount
            running_sum += remaining_amount
            if row['MAX_UNITS'] > 0:
                rows_to_fill.append(idx)
            break
        else:
            break
    # reason for development
    df.loc[rows_to_fill, 'FORECAST_REASON'] = reason
    df_summary = pd.DataFrame({'Reason': [reason], 'Parcels_Available':[len(sdfAvailable)], 'Parcels_Used':[len(rows_to_fill)],
                                'Total_Forecasted_Units': [running_sum], 'Total_Remaining_Units': [target_sum - running_sum]})   
    return df, df_summary  

def forecast_residential_units_infill(df, condition, target_sum, reason):
    # filter to parcels available for development
    sdfAvailable = df.loc[eval(condition)]
    running_sum = 0
    rows_to_fill = []
    # Loop through the rows and fill the 'new_column'
    for idx, row in sdfAvailable.iterrows():
        # Calculate the remaining amount that can be filled
        remaining_amount = target_sum - running_sum
        if row['POTENTIAL_UNITS'] <= remaining_amount:
            # If the current row's value fits, add it to the column
            df.loc[idx, 'FORECASTED_RESIDENTIAL_UNITS'] = row['POTENTIAL_UNITS']
            running_sum += row['POTENTIAL_UNITS']
            if row['POTENTIAL_UNITS'] > 0:
                rows_to_fill.append(idx)
        elif remaining_amount > 0:
            # If it exceeds the remaining amount, fill with the remaining value
            df.loc[idx, 'FORECASTED_RESIDENTIAL_UNITS'] = remaining_amount
            running_sum += remaining_amount
            if row['POTENTIAL_UNITS'] > 0:
                rows_to_fill.append(idx)
            break
        else:
            break
    # reason for development
    df.loc[rows_to_fill, 'FORECAST_REASON'] = reason
    df_summary = pd.DataFrame({'Reason': [reason], 'Parcels_Available':[len(sdfAvailable)], 'Parcels_Used':[len(rows_to_fill)],
                                'Total_Forecasted_Units': [running_sum], 'Total_Remaining_Units': [target_sum - running_sum]})   
    return df, df_summary

def get_target_sum(df, Jurisdiction, Unit_Pool, zoning_type):
    if zoning_type == 'MF':
        return df.loc[(df['Jurisdiction'] == Jurisdiction) & (df['Unit_Pool'] == Unit_Pool), 'Future_Units_Adjusted_MF'].values[0]
    elif zoning_type == 'SF':
        return df.loc[(df['Jurisdiction'] == Jurisdiction) & (df['Unit_Pool'] == Unit_Pool), 'Future_Units_Adjusted_SF'].values[0]
    elif zoning_type == 'Infill':
        return df.loc[(df['Jurisdiction'] == Jurisdiction) & (df['Unit_Pool'] == Unit_Pool), 'Future_Units_Adjusted_Infill'].values[0]
    return df.loc[(df['Jurisdiction'] == Jurisdiction) & (df['Unit_Pool'] == Unit_Pool), 'Future_Units_Adjusted'].values[0]

# funtion to adust the forecasted residential units vs the target sum in the unit pool
def adjust_pools(df, dfPool):
    dfPool_Deductions = df.groupby(['JURISDICTION','FORECAST_REASON'])['FORECASTED_RESIDENTIAL_UNITS'].sum()
    dfPool_Deductions = dfPool_Deductions.reset_index()
    # unit pool field
    dfPool_Deductions['Unit_Pool'] = 'NA'
    # set unit pool to Juirsidiction + Forecast Reason
    dfPool_Deductions['Unit_Pool'] = dfPool_Deductions['JURISDICTION'] + ' ' + dfPool_Deductions['FORECAST_REASON']
    # merge the two dataframes
    dfPool = pd.merge(dfPool, dfPool_Deductions, on='Unit_Pool', how='left')
    # fill NaN with 0
    dfPool['FORECASTED_RESIDENTIAL_UNITS'] = dfPool['FORECASTED_RESIDENTIAL_UNITS'].fillna(0)
    # adjust the forecasted units
    dfPool['Future_Units_Adjusted']    = dfPool['Future_Units_Adjusted'] - dfPool['FORECASTED_RESIDENTIAL_UNITS']
    # adjust the forecasted units
    dfPool['Future_Units_Adjusted_MF'] = dfPool['Future_Units_Adjusted_MF'] - dfPool['FORECASTED_RESIDENTIAL_UNITS']
    # adjust the forecasted units
    dfPool['Future_Units_Adjusted_SF'] = dfPool['Future_Units_Adjusted_SF'] - dfPool['FORECASTED_RESIDENTIAL_UNITS']
    # adjust the forecasted units
    dfPool['Future_Units_Adjusted_Infill'] = dfPool['Future_Units_Adjusted_Infill'] - dfPool['FORECASTED_RESIDENTIAL_UNITS']
    # return the dataframe
    return dfPool_Deductions, dfPool

### Get Data

In [ ]:
# parcel development layer polygons
parcel_db = Path(sdeEdit) / "SDE.Parcel\\SDE.Parcel_History_Attributed"
# query 2022 rows
sdf_units = pd.DataFrame.spatial.from_featureclass(parcel_db)
sdf_units = sdf_units.loc[sdf_units['YEAR'] == 2022]
sdf_units.spatial.sr = sr

# # get parcel level data from Collection SDE
# vhr feature layer polygons 
vhr_db = Path(sdeCollect) / "SDE.Parcel\\SDE.Parcel_VHR"
sdf_vhr = pd.DataFrame.spatial.from_featureclass(vhr_db)
sdf_vhr.spatial.sr = sr
# filter vhr layer to active status
sdf_vhr = sdf_vhr.loc[sdf_vhr['Status'] == 'Active']

# TAZ feature layer polygons
taz_db = Path(sdeBase) / "SDE.Transportation\\SDE.Transportation_Analysis_Zone"
# get as spatial dataframe
sdf_taz = pd.DataFrame.spatial.from_featureclass(taz_db)
# set spatial reference to NAD 1983 UTM Zone 10N
sdf_taz.spatial.sr = sr

# censuse feature class
census_fc    = Path(sdeBase) / "SDE.Census\\SDE.Tahoe_Census_Geography"
# bouns unit boundary feature class
bonus_unit_fc = Path(sdeBase) / "SDE.Planning\SDE.Bonus_unit_boundary"

# disable Z values on block group feature layer
with arcpy.EnvManager(outputZFlag="Disabled"):    
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features="F:\GIS\DB_CONNECT\Vector.sde\SDE.Census\SDE.Tahoe_Census_Geography",
        Output_Geodatabase=r"C:\Users\mbindl\Desktop\Workspace.gdb"
    )
# disable Z values on block group feature layer
with arcpy.EnvManager(outputZFlag="Disabled"):    
    arcpy.conversion.FeatureClassToGeodatabase(
        Input_Features="F:\GIS\DB_CONNECT\Vector.sde\SDE.Planning\SDE.Bonus_unit_boundary",
        Output_Geodatabase=r"C:\Users\mbindl\Desktop\Workspace.gdb"
    )

# block group feature layer polygons with no Z
sdf_block = pd.DataFrame.spatial.from_featureclass(Path(gdb) / 'Tahoe_Census_Geography')
sdf_block = sdf_block.loc[(sdf_block['YEAR'] == 2020) & (sdf_block['GEOGRAPHY'] == 'Block Group')]
sdf_block.spatial.sr = sr

# bonus unit boundary wihtout Z
sdf_bonus = pd.DataFrame.spatial.from_featureclass(Path(gdb) / 'Bonus_unit_boundary')
sdf_bonus.spatial.sr = sr

# get parcel level data from LTinfo
dfIPES       = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelIPESScores/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfLCV_LTinfo = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetParcelsByLandCapability/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfRetired    = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
dfBankedDev  = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfTransacted = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')
dfAllParcels = pd.read_json('https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476')

# get use tables 
# zoning data
sde_engine = get_conn('sde')
with sde_engine.begin() as conn:
    df_uses    = pd.read_sql("SELECT * FROM sde.SDE.PermissibleUses", conn)
    df_special = pd.read_sql("SELECT * FROM sde.SDE.Special_Designation", conn)

### Parcel Data Engineering

In [ ]:
# spatial join to get TAZ
arcpy.SpatialJoin_analysis(sdf_units, sdf_taz, "Existing_Development_TAZ", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join to get Block Group
arcpy.SpatialJoin_analysis(sdf_units, sdf_block, "Existing_Development_BlockGroup", 
                           "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "HAVE_THEIR_CENTER_IN")
# spatial join of Bonus Unit Boundary
arcpy.SpatialJoin_analysis(sdf_units, sdf_bonus, "Existing_Development_BonusUnitBoundary",
                            "JOIN_ONE_TO_ONE", "KEEP_ALL", "", "INTERSECT")

In [ ]:
# spatial dataframe with only initial columns
sdfParcels = sdf_units[initial_columns]

# get results of spatial joins as spatial dataframes
sdf_units_taz   = pd.DataFrame.spatial.from_featureclass("Existing_Development_TAZ", sr=sr)  
sdf_units_block = pd.DataFrame.spatial.from_featureclass("Existing_Development_BlockGroup", sr=sr)
sdf_units_bonus = pd.DataFrame.spatial.from_featureclass("Existing_Development_BonusUnitBoundary", sr=sr)
# cast to string
sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'] = sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'].astype(str)
sdf_units_bonus['WITHIN_BONUSUNIT_BNDY'] = 'No'
# if Id is not NA then within bonus unit boundary = yes, else
sdf_units_bonus.loc[sdf_units_bonus['Id'].notna(), 'WITHIN_BONUSUNIT_BNDY'] = 'Yes'

# map dictionary to sdf_units dataframe to fill in TAZ and Block Group fields
sdfParcels['TAZ']                   = sdfParcels.APN.map(dict(zip(sdf_units_taz.APN,   sdf_units_taz.TAZ_1)))
sdfParcels['BLOCK_GROUP']           = sdfParcels.APN.map(dict(zip(sdf_units_block.APN, sdf_units_block.TRPAID)))
# map IPES score to parcels
sdfParcels['IPES_SCORE']            = sdfParcels['APN'].map(dict(zip(dfIPES.APN, dfIPES.IPESScore)))
sdfParcels['IPES_SCORE_TYPE']       = sdfParcels['APN'].map(dict(zip(dfIPES.APN, dfIPES.IPESScoreType)))
# retired parcels
sdfParcels['RETIRED']               = sdfParcels['APN'].map(dict(zip(dfAllParcels.APN, dfAllParcels.RetiredFromDevelopment)))
sdfParcels['WITHIN_BONUSUNIT_BNDY'] = sdfParcels['APN'].map(dict(zip(sdf_units_bonus.APN, sdf_units_bonus.WITHIN_BONUSUNIT_BNDY)))
# define housnig zoning and density
sdfParcels['HOUSING_ZONING']          = 'NA'
sdfParcels['COMMERCIAL_ALLOWED']      = 'No'
sdfParcels['TOURIST_ALLOWED']         = 'No'

# if the zoning id is in the list of multiple family zones then set to MF
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_sf_mf_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'SF/MF'
# if the zoning id is in the list of single family zones and not in the multiple family zones then set to SF only
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_sf_only_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'SF_only'
# if the zoning id is in the list of multiple family zones and not in the single family zones then set to MF only
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_mf_only_zones(df_uses)['Zoning_ID']), 'HOUSING_ZONING'] = 'MF_only'
# if the zoning id is in the list of commercial zones then set to Commercial
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_commercial_zones(df_uses)['Zoning_ID']), 'COMMERCIAL_ALLOWED'] = 'Yes'
# if the zoning id is in the list of tourist zones then set to Tourist Accommodation
sdfParcels.loc[sdfParcels['ZONING_ID'].isin(get_tourist_zones(df_uses)['Zoning_ID']), 'TOURIST_ALLOWED'] = 'Yes'

# if COUNTY is in EL or PL and SF allowed then set ADU_ALLOWED to yes or if COUNTY is in WA, DG, or CC and parcel acres is greater than 1 and SF allowed then set ADU_ALLOWED to yes
sdfParcels['ADU_ALLOWED'] = 'No'
sdfParcels.loc[(sdfParcels['COUNTY'].isin(['EL','PL'])) & (~sdfParcels['HOUSING_ZONING'].isin(['MF_only', 'NA'])), 'ADU_ALLOWED'] = 'Yes'
sdfParcels.loc[(sdfParcels['COUNTY'].isin(['WA','DG','CC'])) & (sdfParcels['PARCEL_ACRES']>=1) &(~sdfParcels['HOUSING_ZONING'].isin(['MF_only', 'NA'])), 'ADU_ALLOWED'] = 'Yes'

# get density for MF and MF only zones, max residential units, and adjusted residential units
dfMF = get_mf_zones(df_uses)
sdfParcels['DENSITY']                    = sdfParcels['ZONING_ID'].map(dict(zip(dfMF.Zoning_ID, dfMF.Density)))
sdfParcels['MAX_RESIDENTIAL_UNITS']      = sdfParcels['PARCEL_ACRES'] * sdfParcels['DENSITY']
sdfParcels['MAX_UNITS']                  = sdfParcels['MAX_RESIDENTIAL_UNITS']*0.6
sdfParcels['MAX_UNITS']                  = sdfParcels['MAX_UNITS'].fillna(0).astype(int)

# set SF only zones to 1 max unit
sdfParcels.loc[sdfParcels['HOUSING_ZONING'] == 'SF_only', 'MAX_UNITS'] = 1

# set field for underbuilt evaluation
sdfParcels['POTENTIAL_UNITS'] = 0
sdfParcels['POTENTIAL_UNITS'] = sdfParcels['MAX_UNITS'] - sdfParcels['Residential_Units']
# calculate parcels with the greatest buildable potential  filter to the top 10% of parcels
# what value is in the top 10% of the potential buildable units
top_10_threshold = sdfParcels.POTENTIAL_UNITS.quantile(0.9)
# filter out rows where POTENTIAL_BUILDABLE_UNITS is NaN
sdfParcels['TOP_TEN_POTENTIAL_UNITS'] = sdfParcels.apply(lambda x: 'Yes' if x['POTENTIAL_UNITS'] >= top_10_threshold else 'No', axis=1)

# set FORECASTED_RESIDENTIAL_UNITS to 0
sdfParcels['FORECASTED_RESIDENTIAL_UNITS']     = 0
# set FORECAST_COMMERCIAN_UNITS to 0
sdfParcels['FORECASTED_COMMERCIAL_SQFT']       = 0
# set FORECAST_TOURIST_UNITS to 0
sdfParcels['FORECASTED_TOURIST_UNITS']         = 0
# set FORECAST_REASON to na
sdfParcels['FORECAST_REASON']                  = None
# FORECASTED_OCCUPANCY_RATE as a float field
sdfParcels['FORECASTED_RES_OCCUPANCY_RATE']    = 0.0

# export to pickle
sdfParcels.to_pickle(parcel_pickle_part1)
# to feature class
sdfParcels.spatial.to_featureclass(Path(gdb)/'Parcel_Base_2022')

## Model Year Forecasting

### Residential Forecasting

> Methods

1) Assign Known Projects
* Assign using Lookup_Lists\forecast_residential_assigned_units.csv: 
    * Known Residential Allocations from 2023-2024, 
    * Known Residential Bonus Units from permitted projects, 
    * applications in review, and 
    * RBU reservations in LT Info
    * Known Accessory Dwelling permits not completed
    * Remove Known Banking projects that removed units in 2023-2024
* Calcuate Remaining local jurisdiction pool units less units used for known projects
    
2) Assign Residential Bonus Units within Bonus Unit Boundary
* Full build out of CTC Asset Lands using jurisdiction bonus unit pools
* Identify vacant buildable lots within Bonus Unit boundary
* Assign remaining jurisdiction pool units to available parcels within jurisidiction

3) Assigne remaining Jurisdiction Residential Bonus and General Units
* Identify	Vacant buildable lots with allowed Multi-family use, calculate allowed density
* Assign 15% of remaining local jurisdiction Residential Allocation pool units to available multi-family parcels within jurisidiction
* Assign 35% of remaining Banked units to available multi-family parcels (use adjusted weighting from existing residential units?)
* Assign 35% of remaining Converted units to available multi-family parcels (use adjusted weighting from existing residential units?)
    
4) Assign remaining TRPA pool units to available parcels throughout region (use adjusted weighting from existing residential units?)
* Evaluate Vacant Buildable Lots with Single-family Residential Allowed Use	
* Identify	Vacant buildable lots with allowed Single-family use
* Identify	Accessory Dwelling Uses Allowed (All California Parcels and NV Parcels Greater than 1 Acre)
* Evaluate Underbuilt parcels with Multi-family Residential Allowed Use	
* Identify	Underbuilt Residential lots with allowed Multi-family use
* Evaluate Underbuilt parcels with Accessory Dwelling Uses Allowed (All California Parcels and NV Parcels Greater than 1 Acre)	
* Identify parcels that are in Town Centers or within a quarter mile and are top x% of underbuilt parcels. 
* Underbuilt = exclude existing condo and common area land uses. sf mf or mixed use
* ADU potential = exclude existing condo or common area land uses, and wait to use any NV parcels>acre. 


> Get the Parcel Base Data

In [ ]:
# get pickle 1 as a spatially enabled dataframe - spatial joins, foreign keys, and new categorical fields added already
sdfParcels = from_pickle(parcel_pickle_part1)
# Randomly sort parcels so that development can be assigned randomly
sdfParcels = sdfParcels.sample(frac=1).reset_index(drop=True)
# Export index and apn to a csv file for future reference with a name that includes the date and time
# This will allow us to recreate the same random order in the future
sdfParcels[['APN']].to_csv(f"Parcel_Sort_Order_{pd.Timestamp.now().strftime('%Y%m%d%H%M%S')}.csv", index=True)

> Get Lookup Lists

In [ ]:
# path to lookup lists
res_assigned_lookup = "Lookup_Lists/forecast_residential_assigned_units.csv"
res_zoned_lookup    = "Lookup_Lists/forecast_residential_zoned_units.csv"
ctc_assetlands_lookup = "Lookup_Lists/CTC_AssetLands_Lookup.csv"

# get zoned and assigned lookup lists as data frames
dfResZoned    = pd.read_csv(res_zoned_lookup)
dfResAssigned = pd.read_csv(res_assigned_lookup)
# get lookup list for CTC asset lands (17 parcels)
ctc_parcels = pd.read_csv(ctc_assetlands_lookup)

> Assign Known Projects

In [ ]:
# Forecast Known Residential Projects from 2023-2024
# group dfResAssigned by APN and sum Unit Change to aggregate to one total for duplciate 
dfResAssignedGrouped_APN = dfResAssigned.groupby('APN').sum('Unit Change').reset_index()
# assign forecast residential units for assigned projects
sdfParcels['FORECASTED_RESIDENTIAL_UNITS'] = sdfParcels.APN.map(dict(zip(dfResAssignedGrouped_APN.APN, dfResAssignedGrouped_APN['Unit Change'])))
# forecast reason = Assigned for APNs in dfResAssignedGrouped
sdfParcels.loc[sdfParcels['APN'].isin(dfResAssigned['APN']), 'FORECAST_REASON'] = 'Assigned'

In [ ]:
sdfParcels.groupby(['JURISDICTION','FORECAST_REASON'])['FORECASTED_RESIDENTIAL_UNITS'].sum()

In [ ]:
# total forecasted units
total_forecasted_units = sdfParcels['FORECASTED_RESIDENTIAL_UNITS'].sum()
print(f'Total Forecasted Units: {total_forecasted_units}')

> Forecast full buildout of CTC Asset Lands

In [ ]:
## CTC Asset Lands ##
# set the forecast reason to CTC Asset Lands for the 17 parcels n
sdfParcels.loc[(sdfParcels['APN'].isin(ctc_parcels['APN'])) & (sdfParcels['FORECAST_REASON']!='Assigned'), 'FORECAST_REASON'] = 'CTC Asset Lands'
# CTC asset lands that are truly buildable
CTC_condition          = "(sdfParcels['FORECAST_REASON'] == 'CTC Asset Lands') & (sdfParcels['POTENTIAL_UNITS'] > 0) & (sdfParcels['TOP_TEN_POTENTIAL_UNITS'] == 'Yes')"
# assign POTEINTIAL_UNITS to FORECASTED_RESIDENTIAL_UNITS for CTC Asset Lands that are buildable and not built by 2024
sdfParcels.loc[eval(CTC_condition), 'FORECASTED_RESIDENTIAL_UNITS'] = sdfParcels['POTENTIAL_UNITS']

In [ ]:
sdfParcels.groupby(['JURISDICTION','FORECAST_REASON'])['FORECASTED_RESIDENTIAL_UNITS'].sum()

> Subtract Assigned Units from the appropriate pool

In [ ]:
# Subtract Assigned units from the appropriate pool
# group dfResAssigned by Jurisdiction and Unit_Pool
dfGroup_Assigned = dfResAssigned.groupby(['Jurisdiction', 'Unit_Pool']).sum('Unit Change').reset_index()
# drop Occupancy_Rate and Year columns
dfGroup_Assigned = dfGroup_Assigned.drop(columns=['Occupancy_Rate'])
# rename Unit Change to Unit_Change
dfGroup_Assigned = dfGroup_Assigned.rename(columns={'Unit Change':'Unit_Change'})

# merge dfResAssignedGrouped with dfResZoned on Jurisdiction and Pool
dfPool_Assigned = pd.merge(dfResZoned, dfGroup_Assigned, on=['Jurisdiction', 'Unit_Pool'], how='left')

# fill NaN with 0
dfPool_Assigned['Unit_Change'] = dfPool_Assigned['Unit_Change'].fillna(0)
# subtract known project aggregations from the zoned unit pools
dfPool_Assigned['Future_Units_Adjusted'] = dfPool_Assigned['Future_Units'] - dfPool_Assigned['Unit_Change']

In [ ]:
# sum of forecasted units, forecasted units adjusted, and unit change
for i in ['Future_Units', 'Future_Units_Adjusted', 'Unit_Change']:
    print(f"{i}: {dfPool_Assigned[i].sum()}")

> Subtract the CTC asset lands buildout from the appropriate pool

In [ ]:
# filter to CTC Asset Lands parcels 
sdfCTC = sdfParcels.loc[sdfParcels['FORECAST_REASON'] == 'CTC Asset Lands']
# sum of forecasted residential units where forecast reason = ctc asset lands
sdfCTC_ForecastedByJurisdiction = sdfCTC.groupby('JURISDICTION')['FORECASTED_RESIDENTIAL_UNITS'].sum().reset_index()
# set unit pool to Bonus Unit or General based on Jurisdiction
sdfCTC_ForecastedByJurisdiction.loc[(sdfCTC_ForecastedByJurisdiction['JURISDICTION']=='CSLT'), 'Unit_Pool'] = 'General'
sdfCTC_ForecastedByJurisdiction.loc[(sdfCTC_ForecastedByJurisdiction['JURISDICTION']=='PL'), 'Unit_Pool']   = 'Bonus Unit'
sdfCTC_ForecastedByJurisdiction.loc[(sdfCTC_ForecastedByJurisdiction['JURISDICTION']=='EL'), 'Unit_Pool']   = 'General'
# rename columns
sdfCTC_ForecastedByJurisdiction.rename(columns={'JURISDICTION': 'Jurisdiction', 'FORECASTED_RESIDENTIAL_UNITS': 'Forecasted_CTC'}, inplace=True)

# Subtract CTC asset lands from the appropriate pool
dfPool_CTC = sdfCTC_ForecastedByJurisdiction.copy()
# merge with dfResMerge
dfPool_CTC = pd.merge(dfPool_Assigned, dfPool_CTC, on=['Jurisdiction', 'Unit_Pool'], how='left')

# fill NaN with 0
dfPool_CTC['Forecasted_CTC'] = dfPool_CTC['Forecasted_CTC'].fillna(0)
# subtract CTC asset lands from the appropriate pool
dfPool_CTC['Future_Units_Adjusted'] = dfPool_CTC['Future_Units_Adjusted'] - dfPool_CTC['Forecasted_CTC']
# add CTC Max Build to the unit change
dfPool_CTC['Unit_Change'] = dfPool_CTC['Unit_Change'] + dfPool_CTC['Forecasted_CTC']
# drop CTC Built column
dfPool_CTC.drop(columns=['Forecasted_CTC'], inplace=True)

In [ ]:
# sum of forecasted units, forecasted units adjusted, and unit change
for i in ['Future_Units', 'Future_Units_Adjusted', 'Unit_Change']:
    print(f"{i}: {dfPool_CTC[i].sum()}")

In [ ]:
dfPool_CTC

> Setup Unit Pools Proportion to be Used in Each Zone

In [ ]:
dfPool = dfPool_CTC.copy()
# proportion target of forecast development by type
portion_multifamily = .35
portion_singlefamily = .5
portion_infill = .15
# Set units to use for each zoning type
dfPool['Future_Units_Adjusted'] = dfPool['Future_Units_Adjusted'].fillna(0)
dfPool['Future_Units_Adjusted_MF'] = (dfPool['Future_Units_Adjusted'] * portion_multifamily).round().astype(int)
dfPool['Future_Units_Adjusted_SF'] = (dfPool['Future_Units_Adjusted'] * portion_singlefamily).round().astype(int)
dfPool['Future_Units_Adjusted_Infill'] = (dfPool['Future_Units_Adjusted'] * portion_infill).round().astype(int)
# Assign any rounding error to the single family pool
dfPool['Adjustment'] = dfPool['Future_Units_Adjusted'] - dfPool['Future_Units_Adjusted_MF'] - dfPool['Future_Units_Adjusted_SF'] - dfPool['Future_Units_Adjusted_Infill']
dfPool['Future_Units_Adjusted_SF'] = dfPool['Future_Units_Adjusted_SF'] + dfPool['Adjustment']
dfPool.drop(columns=['Adjustment'], inplace=True)

In [ ]:
dfPool

> Define Conditions

In [ ]:
##------------------------------------ Conditional Statements for Forecasting Residential Unit Development------------------------------ ##
# vacant buildable criteria
vacant_buildable_criteria        = "(df['FORECAST_REASON'].isna()) & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['RETIRED'] == 'No') & (df['IPES_SCORE'] > 0)"
placer_vacant_buildable_criteria = "(df['FORECAST_REASON'].isna()) & (df['EXISTING_LANDUSE'] == 'Vacant') & (df['OWNERSHIP_TYPE'] == 'Private') & (df['RETIRED'] == 'No') & (df['IPES_SCORE'] > 726)" 
# Within TRPA Boundary as condition for all
trpa_boundary_criteria = "(df['WITHIN_TRPA_BNDY'] == 1)"
no_zoning_criteria     = "(df['HOUSING_ZONING'] != 'NA')"
sf_only_criteria       = "(df['HOUSING_ZONING'] == 'SF_only')"
mf_only_criteria       = "(df['HOUSING_ZONING'] == 'MF_only')"
sf_mf_criteria         = "(df['HOUSING_ZONING'].isin(['SF/MF', 'MF_only']))"
bonus_criteria         = "(df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')"
adu_criteria           = "(df['ADU_ALLOWED'] == 'Yes')"
towncenter_condition   = "(~df['TOWN_CENTER'].isna())"
tc_quarter_condition   = "(df['LOCATION_TO_TOWNCENTER'] == 'Within 1/4 Mile')"
top_10_condition       = "(df['TOP_TEN_POTENTIAL_UNITS'] == 'Yes')"
parcel_size_condition  = "(df['PARCEL_ACRES'] >= 0.15)"
condo_condition        = "(~df['EXISTING_LANDUSE'].isin('Condominium', 'Condomunium Common Area'))"

> Forecast Jurisdiction Pools

In [ ]:
dfPool = dfPool.copy()
dfPool

In [ ]:
# sum of forecasted units, forecasted units adjusted, and unit change
for i in ['Future_Units', 'Future_Units_Adjusted', 'Unit_Change']:
    print(f"{i}: {dfPool[i].sum()}")

> Assign Units using the Bonus Unit Pools

In [ ]:
#### ------------------------------------------------------UNIT ASSIGNMENTS BY POOL------------------------------------------------------ ####

##-----------------------------------------------------------Bonus Unit Assignments-----------------------------------------------------------##
## CSLT Bonus Unit Assignments ##
# Assign Bonus Units to Multifamily zoned parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'CSLT', 'Bonus Unit', 'MF')
CSLT_Bonus_condition   = "(df['JURISDICTION'] == 'CSLT') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, CSLT_Bonus_condition, target_sum, 'CSLT Bonus Units MF')
df_built_parcels = df_summary
# Assign Bonus Units to Single Family only zoned parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'CSLT', 'Bonus Unit', 'SF')
CSLT_Bonus_condition   = "(df['JURISDICTION'] == 'CSLT') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_only_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, CSLT_Bonus_condition, target_sum, 'CSLT Bonus Units SF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign Bonus Units to Developable Infill parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'CSLT', 'Bonus Unit', 'Infill')
CSLT_Bonus_condition   = "(df['JURISDICTION'] == 'CSLT') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, CSLT_Bonus_condition, target_sum, 'CSLT Bonus Units Infill')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
## Douglas Bonus Unit Assignments ##
# Assign Bonus Units to Multifamily zoned parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'DG', 'Bonus Unit', 'MF')
DG_Bonus_condition     = "(df['JURISDICTION'] == 'DG') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, DG_Bonus_condition, target_sum, 'DG Bonus Units MF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign Bonus Units to Single Family only zoned parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'DG', 'Bonus Unit', 'SF')
DG_Bonus_condition     = "(df['JURISDICTION'] == 'DG') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_only_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, DG_Bonus_condition, target_sum, 'DG Bonus Units SF')
# Assign Bonus Units to Developable Infill parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'DG', 'Bonus Unit', 'Infill')
DG_Bonus_condition     = "(df['JURISDICTION'] == 'DG') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria +  " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, DG_Bonus_condition, target_sum, 'DG Bonus Units Infill')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

## Placer Bonus Unit Assignments ##
# Assign Bonus Units to Multifamily zoned parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'PL', 'Bonus Unit', 'MF')
PL_Bonus_condition     = "(df['JURISDICTION'] == 'PL') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + placer_vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, PL_Bonus_condition, target_sum, 'PL Bonus Units MF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign Bonus Units to Single Family only zoned parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'PL', 'Bonus Unit', 'SF')
PL_Bonus_condition     = "(df['JURISDICTION'] == 'PL') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + placer_vacant_buildable_criteria + " & " + sf_only_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, PL_Bonus_condition, target_sum, 'PL Bonus Units SF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign Bonus Units to Developable Infill parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'PL', 'Bonus Unit', 'Infill')
PL_Bonus_condition     = "(df['JURISDICTION'] == 'PL') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria +  " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, PL_Bonus_condition, target_sum, 'PL Bonus Units Infill')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

## Washoe Bonus Unit Assignments ##
# Assign Bonus Units to Multifamily zoned parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'WA', 'Bonus Unit', 'MF')
WA_Bonus_condition     = "(df['JURISDICTION'] == 'WA') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, WA_Bonus_condition, target_sum, 'WA Bonus Units MF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign Bonus Units to Single Family only zoned parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'WA', 'Bonus Unit', 'SF')
WA_Bonus_condition     = "(df['JURISDICTION'] == 'WA') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_only_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, WA_Bonus_condition, target_sum, 'WA Bonus Units SF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign Bonus Units to Developable Infill parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'WA', 'Bonus Unit', 'Infill')
WA_Bonus_condition     = "(df['JURISDICTION'] == 'WA') & (df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, WA_Bonus_condition, target_sum, 'WA Bonus Units Infill')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

df_built_parcels

In [ ]:
# sum of forecasted units, forecasted units adjusted, and unit change
for i in ['Future_Units', 'Future_Units_Adjusted', 'Unit_Change']:
    print(f"{i}: {dfPool[i].sum()}")

> Assign Units using General Unit Pools

In [ ]:
## ----------------------------------------------------General Unit Assignments---------------------------------------------------- ##
## CSTL General Unit Assignments ##
# Assign General Jurisdiction Units to Multifamily zoned parcels
target_sum             = get_target_sum(dfPool, 'CSLT', 'General', 'MF')
CSLT_MF_condition     = "(df['JURISDICTION'] == 'CSLT') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, CSLT_MF_condition, target_sum, 'CSLT General Units MF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Single Family only zoned parcels
target_sum             = get_target_sum(dfPool, 'CSLT', 'General', 'SF')
CSLT_SF_condition     = "(df['JURISDICTION'] == 'CSLT') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_only_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, CSLT_SF_condition, target_sum, 'CSLT General Units SF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Developable Infill parcels
target_sum             = get_target_sum(dfPool, 'CSLT', 'General', 'Infill')
CSLT_Infill_condition = "(df['JURISDICTION'] == 'CSLT') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, CSLT_Infill_condition, target_sum, 'CSLT General Units Infill')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

## El Dorado General Unit Assignments ##
# Assign General Jurisdiction Units to Multifamily zoned parcels
target_sum             = get_target_sum(dfPool, 'EL', 'General', 'MF')
EL_MF_condition        = "(df['JURISDICTION'] == 'EL') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, EL_MF_condition, target_sum, 'EL General Units MF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Single Family only zoned parcels
target_sum             = get_target_sum(dfPool, 'EL', 'General', 'SF')
EL_SF_condition        = "(df['JURISDICTION'] == 'EL') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_only_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, EL_SF_condition, target_sum, 'EL General Units SF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Developable Infill parcels
target_sum             = get_target_sum(dfPool, 'EL', 'General', 'Infill')
EL_Infill_condition    = "(df['JURISDICTION'] == 'EL') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, EL_Infill_condition, target_sum, 'EL General Units Infill')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

# Placer General Unit Assignments
# Assign General Jurisdiction Units to Multifamily zoned parcels
target_sum             = get_target_sum(dfPool, 'PL', 'General', 'MF')
PL_MF_condition        = "(df['JURISDICTION'] == 'PL') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, PL_MF_condition, target_sum, 'PL General Units MF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Single Family only zoned parcels
target_sum             = get_target_sum(dfPool, 'PL', 'General', 'SF')
PL_SF_condition        = "(df['JURISDICTION'] == 'PL') & " + trpa_boundary_criteria + " & " + placer_vacant_buildable_criteria + " & " + sf_only_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, PL_SF_condition, target_sum, 'PL General Units SF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Developable Infill parcels
target_sum             = get_target_sum(dfPool, 'PL', 'General', 'Infill')
PL_Infill_condition    = "(df['JURISDICTION'] == 'PL') & " + trpa_boundary_criteria + " & " + placer_vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, PL_Infill_condition, target_sum, 'PL General Units Infill')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

# Douglas General Unit Assignments
# Assign General Jurisdiction Units to Multifamily zoned parcels
target_sum             = get_target_sum(dfPool, 'DG', 'General', 'MF')
DG_MF_condition        = "(df['JURISDICTION'] == 'DG') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, DG_MF_condition, target_sum, 'DG General Units MF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Single Family only zoned parcels
target_sum             = get_target_sum(dfPool, 'DG', 'General', 'SF')
DG_SF_condition        = "(df['JURISDICTION'] == 'DG') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_only_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, DG_SF_condition, target_sum, 'DG General Units SF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Developable Infill parcels
target_sum             = get_target_sum(dfPool, 'DG', 'General', 'Infill')
DG_Infill_condition    = "(df['JURISDICTION'] == 'DG') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, DG_Infill_condition, target_sum, 'DG General Units Infill')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

# Washoe General Unit Assignments
# Assign General Jurisdiction Units to Multifamily zoned parcels 
target_sum             = get_target_sum(dfPool, 'WA', 'General', 'MF')
WA_MF_condition        = "(df['JURISDICTION'] == 'WA') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, WA_MF_condition, target_sum, 'WA General Units MF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Single Family only zoned parcels
target_sum             = get_target_sum(dfPool, 'WA', 'General', 'SF')
WA_SF_condition        = "(df['JURISDICTION'] == 'WA') & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_only_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, WA_SF_condition, target_sum, 'WA General Units SF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Developable Infill parcels
target_sum             = get_target_sum(dfPool, 'WA', 'General', 'Infill')
WA_Infill_condition    = "(df['JURISDICTION'] == 'WA') & " + trpa_boundary_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, WA_Infill_condition, target_sum, 'WA General Units Infill')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

In [ ]:
df_built_parcels

> Forecast TRPA Pools

In [ ]:
#### ------------------------------------------------------UNIT ASSIGNMENTS BY POOL------------------------------------------------------ ####
dfPool = dfPool.copy()
##-----------------------------------------------------------Bonus Unit Assignments-----------------------------------------------------------##
## TRPA Bonus Unit Assignments ##
# Assign Bonus Units to Multifamily zoned parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'TRPA', 'Bonus Unit', 'MF')
TRPA_Bonus_condition   = "(df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, TRPA_Bonus_condition, target_sum, 'TRPA Bonus Units MF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign Bonus Units to Single Family only zoned parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'TRPA', 'Bonus Unit', 'SF')
TRPA_Bonus_condition   = "(df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_only_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, TRPA_Bonus_condition, target_sum, 'TRPA Bonus Units SF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign Bonus Units to Developable Infill parcels within the Bonus Unit Boundary
target_sum             = get_target_sum(dfPool, 'TRPA', 'Bonus Unit', 'Infill')
TRPA_Bonus_condition   = "(df['WITHIN_BONUSUNIT_BNDY'] == 'Yes')" + " & " + trpa_boundary_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, TRPA_Bonus_condition, target_sum, 'TRPA Bonus Units Infill')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

## TRPA General Unit Assignments ##
# Assign General Jurisdiction Units to Multifamily zoned parcels
target_sum             = get_target_sum(dfPool, 'TRPA', 'General', 'MF')
TRPA_MF_condition      = trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, TRPA_MF_condition, target_sum, 'TRPA General Units MF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Single Family only zoned parcels
target_sum             = get_target_sum(dfPool, 'TRPA', 'General', 'SF')
TRPA_SF_condition     = trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_only_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, TRPA_SF_condition, target_sum, 'TRPA General Units SF')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)
# Assign General Jurisdiction Units to Developable Infill parcels
target_sum             = get_target_sum(dfPool, 'TRPA', 'General', 'Infill')
TRPA_Infill_condition = trpa_boundary_criteria + " & " + vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, TRPA_Infill_condition, target_sum, 'TRPA General Units Infill')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)


In [ ]:
total = df_built_parcels.Total_Forecasted_Units.sum() + 1162 +54
print(total)

In [ ]:
df_built_parcels.Total_Remaining_Units.sum()

In [ ]:
dfPool.Future_Units_Adjusted.sum()

In [ ]:
df_built_parcels.Total_Remaining_Units.sum()

In [ ]:
dfPool.Future_Units.sum()

In [ ]:
dfPool.Future_Units_Adjusted.sum()

In [ ]:
dfPool.Unit_Change.sum()

In [ ]:
df_built_parcels.Total_Remaining_Units.sum()

In [ ]:
4385-3555

In [ ]:
dfPool.Future_Units.sum()

In [ ]:
dfPool.Future_Units_Adjusted.sum()

In [ ]:
sdfParcels.FORECASTED_RESIDENTIAL_UNITS.sum()

In [ ]:
dfPool.info()
dfPoolMelt = dfPool.melt(id_vars=['Jurisdiction', 'Unit_Pool'], value_vars=['Future_Units_Adjusted_MF', 'Future_Units_Adjusted_SF', 'Future_Units_Adjusted_Infill'])
# drop Future_Units_Adjusted_ from variable
dfPoolMelt['variable'] = dfPoolMelt['variable'].str.replace('Future_Units_Adjusted_', '')

dfPoolMelt['Unit_Pool'] = dfPoolMelt['Jurisdiction'] + ' ' + dfPoolMelt['Unit_Pool']
dfPoolMelt.rename(columns={'variable': 'Reason', 'value': 'Units'}, inplace=True)

In [ ]:
dfForecastGroup = sdfParcels.groupby(['FORECAST_REASON'])['FORECASTED_RESIDENTIAL_UNITS'].sum().reset_index()
# split last FORECAST Reason into two columns based on last space in string
dfForecastGroup['Reason'] = dfForecastGroup['FORECAST_REASON'].str.split(' ').str[-1]
dfForecastGroup['Jurisdiction'] = dfForecastGroup['FORECAST_REASON'].str.split(' ').str[0]
# if FORECAST_REASON valie contains 'Bonus' then set Unit Pool to Bonus Units
dfForecastGroup.loc[dfForecastGroup['FORECAST_REASON'].str.contains('Bonus'), 'Unit_Pool'] = dfForecastGroup.Jurisdiction + ' ' + 'Bonus Unit'
# if FORECAST_REASON valie contains 'General' then set Unit Pool to General Units
dfForecastGroup.loc[dfForecastGroup['FORECAST_REASON'].str.contains('General'), 'Unit_Pool'] = dfForecastGroup.Jurisdiction + ' ' + 'General'


In [ ]:
dfMerge = dfForecastGroup.merge(dfPoolMelt, on=['Unit_Pool', 'Reason'], how='left')
dfMerge['Unit_Diff'] = dfMerge['FORECASTED_RESIDENTIAL_UNITS'] - dfMerge['Units']
dfMerge

In [ ]:
# sdfParcels where forecast reason is CLT Bonus Units MF
sdfParcels.loc[sdfParcels['FORECAST_REASON'] == 'CSLT Bonus Units MF']

In [ ]:
dfMerge

> Assign the Remainders

In [ ]:
# Assign Units to SF/MF and MF only parcels, 35% of all pools to MF only parcels
# Check total units available
# Assign Units to SF only parcels, 50% of all pools to SF only parcels
# Assign Units to underbuilt MF parcels within the Bonus Unit Boundary
# Assign Units to parcels within a Town Center
# Assign Units to parcels within 1/4 mile of a Town Center
## TRPA ADU Unit Assignments ##
# Assign remaining Units to parcels eligible for ADU constuction


In [ ]:
# assign ADUs to existing residential parcels residential units=1 and ADU_ALLOWED = Yes
target_sum = get_target_sum(dfPool, 'TRPA', 'General', 'ADU')
TRPA_ADU_condition = trpa_boundary_criteria + " & " + adu_criteria
sdfParcels, df_summary = forecast_residential_units(sdfParcels, TRPA_ADU_condition, target_sum, 'TRPA ADU Units')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

# assign the rest of units to Town Center infill
target_sum = get_target_sum(dfPool, 'TRPA', 'General', 'Infill')
TRPA_Infill_condition = trpa_boundary_criteria + " & " + towncenter_condition + " & "+ vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, TRPA_Infill_condition, target_sum, 'TRPA Town Center Infill Units')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

# assign the remainder to within a quarter mile of town center
target_sum = get_target_sum(dfPool, 'TRPA', 'General', 'Infill')
TRPA_Infill_condition = trpa_boundary_criteria + " & " + tc_quarter_condition + " & "+ vacant_buildable_criteria + " & " + sf_mf_criteria
sdfParcels, df_summary = forecast_residential_units_infill(sdfParcels, TRPA_Infill_condition, target_sum, 'TRPA Quarter Mile Infill Units')
df_built_parcels = pd.concat([df_built_parcels, df_summary], ignore_index=True)

> Assign Forecast Year

In [ ]:
## Assigning Development Year to Parcels ##
# Get total development by 2035
TotalDevelopment = dfResZoned.Future_Units.sum()
Development_2035 = (TotalDevelopment*.46).astype(int)
sdfParcels['Development_Year']=None
#Assining all development that's currently in the works as being cone by 2035
sdfParcels.loc[sdfParcels['FORECAST_REASON'] == 'Assigned', 'Development_Year'] = 2035
RemainingDevelopment_2035 = Development_2035 - sdfParcels.loc[sdfParcels['FORECAST_REASON'] == 'Assigned', 'FORECASTED_RESIDENTIAL_UNITS'].sum()

Development_2035_Condition = sdfParcels['FORECASTED_RESIDENTIAL_UNITS'].where(sdfParcels['FORECAST_REASON'] != 'Assigned').cumsum()
sdfParcels.loc[Development_2035_Condition < RemainingDevelopment_2035, 'Development_Year'] = 2035
sdfParcels.loc[(sdfParcels['FORECAST_REASON']!= '') & (sdfParcels['Development_Year'].isnull()), 'Development_Year'] = 2050
development_year = sdfParcels.groupby('Development_Year')['FORECASTED_RESIDENTIAL_UNITS'].sum()
development_year

#### Assign Occupancy Rate to all forecasted residential parcels

In [ ]:
sdfParcels['FORECASTED_OCCUPANCY_RATE'] = 0
# map lookup known project occupancy rates to parcels
sdfParcels['FORECASETED_OCCUPANCY_RATE'] = sdfParcels['APN'].map(dict(zip(dfResAssigned.APN, dfResAssigned['Occupancy_Rate'])))
sdfParcels.FORECAST_REASON.value_counts()

# if FORECAST_REASON has the word 'Bonus' in it set occupancy rate to 100%
sdfParcels.loc[sdfParcels['FORECAST_REASON'].fillna('').str.contains('Bonus'), 'FORECASTED_OCCUPANCY_RATE'] = 1
# if FORECAST_REASON has the word 'CTC' in it set occupancy rate to 100%
sdfParcels.loc[sdfParcels['FORECAST_REASON'].fillna('').str.contains('CTC'), 'FORECASTED_OCCUPANCY_RATE'] = 1
# if FORECAST_REASON has the word 'General' and housing zoning is SF_only in it set occupancy rate to 35
sdfParcels.loc[(sdfParcels['FORECAST_REASON'].fillna('').str.contains('General')) & (sdfParcels['HOUSING_ZONING'] == 'SF_only'), 'FORECASTED_OCCUPANCY_RATE'] = 0.35

### Summary

In [ ]:
built_units = sdfParcels.groupby('FORECAST_REASON').agg({'FORECASTED_RESIDENTIAL_UNITS':'sum'})
dfResZoned = dfPool.copy()
# Create a dictionary to map the forecast reason to the Jurisdiction and Unit Pool
Forecast_Reason_lookup = {'CSLT Bonus Units Built':{'Jurisdiction':'CSLT', 'Unit_Pool':'Bonus Unit'},
                          'CSLT General Units Built':{'Jurisdiction':'CSLT', 'Unit_Pool':'General'},
                          'EL General Units Built':{'Jurisdiction':'EL', 'Unit_Pool':'General'},
                          'PL Bonus Units Built':{'Jurisdiction':'PL', 'Unit_Pool':'Bonus Unit'},
                          'PL General Units Built':{'Jurisdiction':'PL', 'Unit_Pool':'General'},
                          'WA Bonus Units Built':{'Jurisdiction':'WA', 'Unit_Pool':'Bonus Unit'},
                          'WA General Units Built':{'Jurisdiction':'WA', 'Unit_Pool':'General'},
                          'DG Bonus Units Built':{'Jurisdiction':'DG', 'Unit_Pool':'Bonus Unit'},
                          'DG General Units Built':{'Jurisdiction':'DG', 'Unit_Pool':'General'},
                          'TRPA Bonus Units Built':{'Jurisdiction':'TRPA', 'Unit_Pool':'Bonus Unit'},
                          'TRPA General Units Built':{'Jurisdiction':'TRPA', 'Unit_Pool':'General'},
                          'ADU Units Built':{'Jurisdiction':'TRPA', 'Unit_Pool':'ADU'}}
# Map 'Jurisdiction' and 'Unit_Pool' separately from the dictionary
built_units['Jurisdiction'] = built_units.index.map(lambda x: Forecast_Reason_lookup.get(x, {}).get('Jurisdiction'))
built_units['Unit_Pool'] = built_units.index.map(lambda x: Forecast_Reason_lookup.get(x, {}).get('Unit_Pool'))
unit_comparison = built_units.merge(dfResZoned, how='left', on=['Jurisdiction', 'Unit_Pool'])
unit_comparison['Difference'] = unit_comparison['Future_Units_Adjusted'] - unit_comparison['FORECASTED_RESIDENTIAL_UNITS']
unit_comparison.to_csv('unit_comparison.csv', index=False)

In [ ]:
# forecasted residential uints by location to twon center
sdfParcels.groupby(['LOCATION_TO_TOWNCENTER','FORECAST_REASON'])['FORECASTED_RESIDENTIAL_UNITS'].sum()

In [ ]:
# Summarize Existing and Forecasted Units by Jurisdiction and Unit Pool by TAZ
dfTAZ = sdfParcels.groupby('TAZ')[['FORECASTED_RESIDENTIAL_UNITS', 'Residential_Units']].sum().reset_index()
dfTAZ.to_csv(data_dir/'TAZ_Units.csv', index=False)


In [ ]:
# gropu by TAZ and sum forecasted residential units and residential units
# Forecast year total residential units by TAZ
sdfParcels.groupby('TAZ')[['FORECASTED_RESIDENTIAL_UNITS', 'Residential_Units']].sum().reset_index()
# total units with forecast year 2035 and 2050
sdfParcels.groupby('Development_Year')[['FORECASTED_RESIDENTIAL_UNITS', 'Residential_Units']].sum().reset_index()

### Tourist Accommodation Forecast

In [ ]:
# lookup lists
tau_assigned_lookup = "Lookup_Lists/forecast_tourist_assigned_units.csv"
# get assigned units lookup as data frames
dfTouristAssigned = pd.read_csv(tau_assigned_lookup)
# assign tourist units to parcels
sdfParcels['FORECASTED_TAU_UNITS'] = 0
# set tourist units to assigned total
sdfParcels['FORECASTED_TAU_UNITS'] = sdfParcels.APN.map(dict(zip(dfTouristAssigned.APN, dfTouristAssigned['Unit_Pool'])))

### Commercial Floor Area Forecast

In [ ]:
# lookup lists
cfa_assigned_lookup = "Lookup_Lists/forecast_commercial_assigned_units.csv"
# get zoned units lookups as data frames
dfCommercialAssigned = pd.read_csv(cfa_assigned_lookup)
# set commercial floor area to assigned total
sdfParcels['FORECASTED_COMMERCIAL_FLOOR_AREA'] = 0
sdfParcels['FORECASTED_COMMERCIAL_FLOOR_AREA'] = sdfParcels.APN.map(dict(zip(dfCommercialAssigned.APN, dfCommercialAssigned['NEW_CFA'])))

### Exports

In [ ]:
# export to pickle
sdfParcels.to_pickle(parcel_pickle_part2)
# to csv
sdfParcels.to_csv(data_dir/'Parcels_Forecast.csv', index=False)
# to feature class
sdfParcels.spatial.to_featureclass(Path(gdb)/'Parcel_Forecast')